In [ ]:
# The 'os' module gives us all the tools we need to search in the file system
import os
# Magic to show the plots within the notebook:
%matplotlib inline
# Import libraries to be used
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv as csv
import glob
import time

In [ ]:
t=time.process_time()
path = "C:\\Users\\Lenovo\\Desktop\\Police - Extracted"
csvFiles = glob.glob(path + "\\*-*-street.csv")

csvlist = []

for files in csvFiles:
     raw_data_df = pd.read_csv(files, index_col=None, header=0)
     csvlist.append(raw_data_df)

df = pd.concat(csvlist, ignore_index=True)
elapsed_time=time.process_time()-t
print("Opening and combining raw data in a single data frame took %.2f seconds for %d csv files."
       % (elapsed_time, len(csvlist)))

In [ ]:
df.info()

In [ ]:
del df['Context']
df = df.dropna(subset=['Last outcome category','LSOA code','LSOA name','Longitude','Latitude','Crime ID'])

In [ ]:
df.info()

In [ ]:
df['Latitude']=round(df['Latitude'],6)
df['Longitude']=round(df['Longitude'],6)

In [ ]:
df['Records']=1

In [ ]:
df['Position'] = list(zip(df['Latitude'],df['Longitude']))

In [ ]:
location_raw_data = {'Latitude': df['Latitude'],
                     'Longitude': df['Longitude']} 

location_df = pd.DataFrame(location_raw_data, columns = ['Latitude', 'Longitude'])

location_df['Position'] = list(zip(location_df['Latitude'],location_df['Longitude']))

location_df = location_df.drop_duplicates()

location_df = location_df.reset_index(drop=True)

Nodenumber = []
for i in range(len(location_df)): 
  Nodenumber.append(i+1)

location_df['Nodenumber'] = pd.Series(Nodenumber, index=location_df.index)

location_df['Node_id'] = location_df.apply(lambda x:'%s-N' % (x['Nodenumber']),axis=1)

In [ ]:
nodes = dict(zip(location_df['Position'], location_df['Node_id']))
df['Node_id'] = df['Position'].map(nodes)

In [ ]:
df.info()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head(2)

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import gmplot

In [ ]:
import plotly.plotly as py
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

In [ ]:
map_raw_data = {'Latitude': df['Latitude'],
                     'Longitude': df['Longitude']} 

map_df = pd.DataFrame(map_raw_data, columns = ['Latitude', 'Longitude'])

map_df = map_df.drop_duplicates()

map_df = map_df.reset_index(drop=True)

map_df['Position'] = list(zip(map_df['Latitude'],map_df['Longitude']))

nodes = dict(zip(location_df['Position'], location_df['Node_id']))
map_df['Node_id'] = map_df['Position'].map(nodes)

In [ ]:
t=time.process_time()
map_df['Count'] = df.groupby(['Node_id'])['Records'].sum().values
elapsed_time=time.process_time()-t
print("Bringing how many crime records there are for every position node took %.2f seconds for %d nodes."
       % (elapsed_time, len(map_df)))
map_df